In [1]:
import scanpy as sc
import numpy as np
import torch

import os,sys
pwd = '/home/lijiahao/workbench/SAVE/'
sys.path.append(pwd)
os.chdir(pwd)

from model.save_model import SAVE
from model.utils.process_h5ad import batch_scale, preprocessing_rna

In [2]:
# download the data via link: https://figshare.com/ndownloader/files/24539828

In [3]:
adata = sc.read_h5ad('./data/PBMC_10K_results.h5ad')

In [4]:
adata.X = adata.layers['X_log1p']

In [5]:
adata, _ = batch_scale(adata)

batch_scale: 100%|████████████████████████████████| 1/1 [00:00<00:00,  3.16it/s]


In [6]:
adata.obs['batch'] = adata.obs['batch'].astype('category')

In [11]:
device = torch.device('cuda:0')
seed = 1202
kwargs = {
    "device": device,
    "seed": seed,
    "is_data_scaled": True,
}
import yaml

with open(pwd + "/model_setting.yml", "r") as f:
    setting = yaml.safe_load(f)

kwargs.update(setting["SAVE-B"]["train"])
kwargs.update(setting["SAVE-B"]["model"])

In [ ]:
from model.save_model import SAVE

save_model = SAVE(
    adata=adata.copy(),
    is_initialized=True,
    condition_cols=['batch'],
    **kwargs,
    )


In [ ]:
save_model.train(**kwargs)
save_model.save_ckpt('ckpt/PBMC_10K.pt')

In [ ]:
save_model.load_ckpt('ckpt/PBMC_10K.pt')

In [ ]:
latent = save_model.get_latent()

In [16]:
adata_test = sc.read_h5ad('./data/PBMC_10K_test.h5ad')
adata_test.X = adata_test.layers['X_log1p']
adata_test.obs['batch'] = adata_test.obs['batch'].astype('category')

In [ ]:
adata_test, _ = batch_scale(adata_test)

In [ ]:
save_model.zero_shot_embeding(input_adata=adata_test, is_data_scaled=True, batch_size=256)

In [ ]:
adata_test.obs

In [ ]:
sc.set_figure_params(dpi=100)
sc.pp.neighbors(adata_test, use_rep="SAVE")
sc.tl.umap(adata_test)
sc.pl.umap(adata_test, color=['batch', 'celltype'])

In [32]:
adata_whole = sc.concat([adata, adata_test])

In [ ]:
save_model.device = torch.device('cpu')
save_model.zero_shot_embeding(adata_whole, is_data_scaled=True)

### scIB evaluation

In [7]:
from scib_metrics.benchmark import Benchmarker

In [8]:
sc.pp.pca(adata)
adata.obsm['Unintegraged'] = adata.obsm['X_pca']

In [10]:
import scib
results = scib.metrics.metrics(
    adata,
    adata_int=adata,
    batch_key='batch',
    label_key='celltype',
    embed="X_scGPT",
    isolated_labels_asw_=False,
    silhouette_=True,
    hvg_score_=False,
    graph_conn_=True,
    pcr_=True,
    isolated_labels_f1_=False,
    trajectory_=False,
    nmi_=True,  # use the clustering, bias to the best matching
    ari_=True,  # use the clustering, bias to the best matching
    cell_cycle_=False,
    kBET_=False,  # kBET return nan sometimes, need to examine
    ilisi_=False,
    clisi_=False,
)

/mnt/sdc/lijiahao/miniconda3/envs/SAVE/lib/python3.9/site-packages/scib/metrics/metrics.py:293: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  res_max, nmi_max, nmi_all = opt_louvain(


NMI...
ARI...
Silhouette score...
PC regression...
Graph connectivity...


In [11]:
results

,0
NMI_cluster/label,0.835043
ARI_cluster/label,0.866458
ASW_label,0.754482
ASW_label/batch,0.949802
PCR_batch,0.647313
cell_cycle_conservation,NaN
isolated_label_F1,NaN
isolated_label_silhouette,NaN
graph_conn,0.932286
kBET,NaN
